# ch09 텐서플로 시작하기

- 파이썬으로 수행할 계산 그래프(computation graph)를 정의한 다음, 텐서플로가 최적화된 C++코드를 사용해 이 그래프를 효율적으로 실행시킨다.

In [1]:
import tensorflow as tf

/Users/angelx/.pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
print(tf.__version__)

1.10.1


In [3]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name='y')
f = x*x*y + y +2

- 실제로 어떤 계산도 수행하지 않는다.
- 단지 계산 그래프만 만들 뿐이다.
- 변수조차도 초기화되지 않는다.
- 이 계산 그래프를 평가하려면 텐서플로 세션을 시작하고 변수를 초기화한다음 f를 평가해야 한다.
- 텐서플로 세션은 연산은 CPU나 GPU같은 장치에 올리고 실행하는 것을 도와주며 모든 변수값을 가지고 있다.

- 다음 코드는 세션을 만들고 변수를 초기화하고 f를 평가한 후 세션을 닫는다.

In [4]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

42


In [5]:
sess.close()

- sess.run()을 반복하면 번거롭다. 간소화.

In [6]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

- 각 변수의 초기화를 일일이 실행하는 대신 global_variable_initializer() 함수를 사용할 수 있다.
- 이 함수를 초기화를 바로 수행하지 않고 계산 그래프가 실행될 때 모든 변수를 초기화할 노드를 생성한다.

In [7]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
    print(result)

42


- 주피터나 파이썬 셸에서는 InteractiveSession을 만드는 편이 편리하다.
- 일반적인 Session과 다른 점은 InteractiveSession이 만들어질 때 자동으로 자신을 기본 세션으로 지정한다는 점
- with 블록을 사용할 필요가 없다.
- 수동으로 세션을 종료해야 한다.

In [8]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

42


In [9]:
sess.close()

- 일반적으로 텐서플로 프로그램은 두 부분으로 나뉜다.
- 계산 그래프를 만드는 구성단계
- 그래프를 실행하는 실행단계

- 구성 단계에서는 훈련에 필요한 계산고 머신러닝 모델을 표현한 계산 그래프를 만든다.
- 실행 단계에서는 훈련 스탭을 반복해서 평가하고 모델 파라미터를 점진적으로 개선하기 위해 반복 루프를 수행한다.

## 9.3 계산 그래프 관리

- 노드를 만들면 자동으로 기본 계산 그래프에 추가된다.

In [10]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

- 가끔 독립적인 계산 그래프를 여러 개 만들어야 할 때가 있다.
- 새로운 Graph 객체를 만들어 with 블록 안에서 임시로 기본 계산 그래프로 사용한다.

In [11]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [12]:
x2.graph is graph

True

In [13]:
x2.graph is tf.get_default_graph()

False

- jupyter에서 같은 명령을 여러 번 실행하는 경우가 많다.
- 기본 그래프에 중복된 노드가 많이 포함된다.
- jupyter 커널을 다시 시작하는 것이 한 가지 해법
- 더 편리한 방법은 tf.reset_default_graph()로 기본 그래프를 초기화해주는 것

## 9.4 노드 값의 생애주기

- 한 노드를 평가할 때 텐서플로는 이 노드가 의존하고 있는 다른 노드들을 찾아 자동으로 먼저 평가한다.

In [14]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


- 이 코드는 y와 z를 평가하기 위해 w와 x를 두 번 평가한다.

- 모든 노드의 값은 계산 그래프 실행 간에 유지되지 않는다.
- 변수값은 초기화되어 세션이 종료될 때까지 남아 있다.

- w와 x를 두 번 평가하지 않고 y와 z를 효율적으로 평가하려면 텐서플로가 한 번의 그래프 실행에서 y와 z를 모두 평가하도록 만들어야 한다.

In [15]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


- 단일 프로세스 텐서플로에서는 같은 그래프를 재사용하더라도 여러 세션에서 어떤 상태도 공유하지 않는다.
- 각 세션은 모든 변수에 대해 고유한 본사본을 가진다.
- 분산 텐서플로에서는 변수 상태가 세션이 아니라 서버에 저장되므로 여러 세션이 같은 변수를 공유할 수 있다.